In [1]:
import pandas as pd
from sklearn.datasets import load_iris

In [2]:
iris = load_iris(as_frame=True)["data"]
target = load_iris(as_frame=True)["target"]
data = pd.concat([target, iris], axis=1)

In [3]:
data.head()

,target,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,0,5.1,3.5,1.4,0.2
1,0,4.9,3.0,1.4,0.2
2,0,4.7,3.2,1.3,0.2
3,0,4.6,3.1,1.5,0.2
4,0,5.0,3.6,1.4,0.2


In [4]:
filepath_predictors = "../data/predictors.csv"
iris.to_csv(filepath_predictors, index=False, header=False)

In [5]:
filepath_data = "../data/data.csv"
data.to_csv(filepath_data, index=False, header=False)

In [6]:
import sagemaker
from sagemaker.image_uris import retrieve
from sagemaker.inputs import TrainingInput
from sagemaker.estimator import Estimator

In [7]:
session = sagemaker.Session()

In [8]:
region_name = session.boto_region_name
region_name

'us-east-2'

In [9]:
container_image = retrieve(framework='xgboost', region=region_name, version='latest')
container_image

'825641698319.dkr.ecr.us-east-2.amazonaws.com/xgboost:latest'

In [10]:
bucket = session.default_bucket()
bucket

'sagemaker-us-east-2-735421146733'

In [11]:
uploaded_data = session.upload_data(path=filepath_data, bucket=bucket, key_prefix='training')
uploaded_data

's3://sagemaker-us-east-2-735421146733/training/data.csv'

In [12]:
input_data = TrainingInput(s3_data=uploaded_data, content_type='text/csv')

In [13]:
role = "arn:aws:iam::735421146733:role/AmazonSageMaker-ExecutionRole"

In [14]:
xgboost = Estimator(image_uri=container_image,
                    role=role,
                    instance_type='ml.m5.large',
                    instance_count=1,
                    output_path=f"s3://{bucket}/output",
                    sagemaker_session=session)

In [15]:
xgboost.set_hyperparameters(num_round=5, max_depth=5)

In [16]:
xgboost.fit({'train': input_data})

2021-07-10 08:09:09 Starting - Starting the training job...
2021-07-10 08:09:31 Starting - Launching requested ML instancesProfilerReport-1625904541: InProgress
......
2021-07-10 08:10:31 Starting - Preparing the instances for training......
2021-07-10 08:11:52 Downloading - Downloading input data...
2021-07-10 08:12:23 Training - Training image download completed. Training in progress.
2021-07-10 08:12:23 Uploading - Uploading generated training model.Arguments: train
[2021-07-10:08:12:17:INFO] Running standalone xgboost training.
[2021-07-10:08:12:17:INFO] Path /opt/ml/input/data/validation does not exist!
[2021-07-10:08:12:17:INFO] File size need to be processed in the node: 0.0mb. Available memory size in the node: 187.46mb
[2021-07-10:08:12:17:INFO] Determined delimiter of CSV input is ','
[08:12:17] S3DistributionType set as FullyReplicated
[08:12:17] 150x4 matrix with 600 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[08:12:17] src/tree/updat

In [17]:
xgboost.latest_training_job.job_name

'xgboost-2021-07-10-08-09-01-283'

In [18]:
xgboost.model_data

's3://sagemaker-us-east-2-735421146733/output/xgboost-2021-07-10-08-09-01-283/output/model.tar.gz'

In [47]:
import boto3

In [54]:
runtime = boto3.client("sagemaker-runtime")

In [55]:
with open(filepath_predictors, "r") as f:
    dados = f.read()

In [58]:
EndpointName = "Endpoint-ZGqf99QBmhaK"

In [59]:
response = runtime.invoke_endpoint(EndpointName=EndpointName, Body=dados)

ValidationError: An error occurred (ValidationError) when calling the InvokeEndpoint operation: Endpoint Endpoint-ZGqf99QBmhaK of account 735421146733 not found.